# prepare_dataset

Track Zeno's face images using an [exisiting tool](https://github.com/IntelligentBehaviourUnderstandingGroup/dlib_and_chehra_stuff) and prepare them for manual annotation.

## Step 1: Import modules

In [1]:
import os
import sys
import cv2
import glob
import time
import shutil
import numpy as np
try:
    from ConfigParser import ConfigParser    # If using Python 2.7
except ImportError:
    from configparser import ConfigParser    # If using Python 3.5
config = ConfigParser()
config.read('prepare_dataset.ini')
sys.path.append(os.path.realpath(config.get('facial_landmark_tracker', 'repository_path')))
import ibug_face_tracker
from zeno_face_tracker_helpers import *
print('All modules imported.')

All modules imported.


## Step 2: Initialise the landmark localiser

In [3]:
tracker = ibug_face_tracker.FaceTracker(os.path.realpath(config.get('facial_landmark_tracker', 
                                                                    'ert_model_path')), 
                                        os.path.realpath(config.get('facial_landmark_tracker', 
                                                                    'auxiliary_model_path')))
tracker.face_detection_scale = config.getfloat('facial_landmark_tracker', 'face_detection_scale')
tracker.minimum_face_size = config.getint('facial_landmark_tracker', 'minimum_face_size')
tracker.hard_failure_threshold = -1e6
tracker.estimate_head_pose = False
tracker.eye_iterations = 0
print('Facial landmark localiser initialised.')

Facial landmark localiser initialised.


## Step 3: Localise Facial Landmarks

In [4]:
# Enumerate jobs
source_images = sorted(glob.glob(os.path.realpath(os.path.join('./dataset', 'cam*', '*.png'))))
source_images = [x for x in source_images if '.pts.png' not in x]
print('%d images to be processed.' % len(source_images))

# Get landmarks
successes = 0
last_check_time = time.time()
for idx, image_path in enumerate(source_images):
    image = cv2.imread(image_path)
    tracker.reset()
    tracker.track(image)
    if tracker.has_facial_landmarks:
        pts_path = os.path.splitext(image_path)[0] + '.init.pts'
        save_pts(pts_path, tracker.facial_landmarks)
        tracker.plot_current_result(image)
        rendering_path = pts_path + '.png'
        cv2.imwrite(rendering_path, image)
        successes += 1
    current_time = time.time()
    if current_time - last_check_time > 10.0:
        last_check_time = current_time
        print('%d images have been processed.' % (idx + 1))
print('Done, all %d images have been processed, %d of which have landmarks localised.' % 
      (len(source_images), successes))

1200 images to be processed.
112 images have been processed.
222 images have been processed.
341 images have been processed.
462 images have been processed.
579 images have been processed.
682 images have been processed.
795 images have been processed.
905 images have been processed.
1012 images have been processed.
1123 images have been processed.
Done, all 1200 images have been processed, 1034 of which have landmarks localised.


## Step 4: Prepare the annotation jobs than can be loaded in FLAT

In [2]:
# Enumerate the recording sessions
recording_sessions = sorted(glob.glob(os.path.realpath(os.path.join('./dataset', 'cam*'))))
recording_sessions = [x for x in recording_sessions if os.path.isdir(x)]

# How many examples per session?
num_samples = 1e6
for session in recording_sessions:
    images = glob.glob(os.path.join(session, '*.png'))
    images = [x for x in images if '.pts.' not in x]
    num_samples = min(num_samples, len(images))

# Permutate samples in the sessions so that each annotation batch gets a bit of everything
indices = np.zeros((len(recording_sessions), num_samples), dtype=int)
for idx in range(indices.shape[1]):
    indices[:, idx] = np.random.permutation(indices.shape[0])

# Create jobs
for batch_idx, batch in enumerate(indices):
    batch_content = []
    for idx, session in enumerate(batch):
        image_path = os.path.join(recording_sessions[session], '%06d.png' % idx)
        pts_path = os.path.splitext(image_path)[0] + '.init.pts'
        if os.path.exists(pts_path):
            pts = load_pts(pts_path)
            if pts.shape[0] == 68:
                
                batch_content.append((image_path, pts))
    job_path = os.path.realpath(os.path.join('./dataset', 'batch_%03d.fad' % batch_idx))
    duplication = 0
    while True:
        if os.path.exists(job_path):
            duplication += 1
            job_path = os.path.realpath(os.path.join(
                './dataset', 'batch_%03d.%03d.fad' % (batch_idx, duplication)))
        else:
            break
    save_annotation_job(job_path, batch_content, 68)
    print('Annotation job created: ' + job_path)
    backup_path = job_path + '.bak'
    shutil.copyfile(job_path, backup_path)
    print('Backup file created: ' + backup_path)

Annotation job created: D:\hhj\zeno_face_tracker\dataset\batch_000.fad
Backup file created: D:\hhj\zeno_face_tracker\dataset\batch_000.fad.bak
Annotation job created: D:\hhj\zeno_face_tracker\dataset\batch_001.fad
Backup file created: D:\hhj\zeno_face_tracker\dataset\batch_001.fad.bak
Annotation job created: D:\hhj\zeno_face_tracker\dataset\batch_002.fad
Backup file created: D:\hhj\zeno_face_tracker\dataset\batch_002.fad.bak
Annotation job created: D:\hhj\zeno_face_tracker\dataset\batch_003.fad
Backup file created: D:\hhj\zeno_face_tracker\dataset\batch_003.fad.bak
Annotation job created: D:\hhj\zeno_face_tracker\dataset\batch_004.fad
Backup file created: D:\hhj\zeno_face_tracker\dataset\batch_004.fad.bak
Annotation job created: D:\hhj\zeno_face_tracker\dataset\batch_005.fad
Backup file created: D:\hhj\zeno_face_tracker\dataset\batch_005.fad.bak
Annotation job created: D:\hhj\zeno_face_tracker\dataset\batch_006.fad
Backup file created: D:\hhj\zeno_face_tracker\dataset\batch_006.fad.bak